# 1. Thư viện

In [1]:
import os
import json
import re

from dotenv import load_dotenv
from langchain_qdrant import Qdrant, RetrievalMode
from qdrant_client.models import Filter, FieldCondition, MatchValue
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

d:\Download_Python\lib\importlib\__init__.py:127: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


# 2. Thực thi

## 2.1. Mô hình embedding

### 2.1. Mô hình bkai

In [2]:
EMBEDDINGS_MODEL_bkai = "bkai-foundation-models/vietnamese-bi-encoder"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

embeddings_bkai = HuggingFaceBgeEmbeddings(
    model_name=EMBEDDINGS_MODEL_bkai,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

### 2.2. Mô hình google-bert/bert-base-multilingual-cased

In [3]:
EMBEDDINGS_MODEL_google_bert = "google-bert/bert-base-multilingual-cased"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

embeddings_google_bert = HuggingFaceBgeEmbeddings(
    model_name=EMBEDDINGS_MODEL_google_bert,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

No sentence-transformers model found with name google-bert/bert-base-multilingual-cased. Creating a new one with mean pooling.
d:\Download_Python\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## 2.2. Kết nối đến collection

In [4]:
URL_QDRANT_2 = os.getenv("URL_QDRANT_2")
API_QDRANT_2 = os.getenv("API_QDRANT_2")

### 2.2.1. Collection "semantic_Luat_bkai"

In [5]:
exist_Qdrant_noKeys = Qdrant.from_existing_collection(
    embedding = embeddings_bkai,
    url = URL_QDRANT_2,
    api_key = API_QDRANT_2,
    prefer_grpc=True,
    collection_name = "semantic_Luat_bkai",
	metadata_payload_key="metadata"
)

### 2.2.2. Collection "semantic_Luat_bkai_More_Keywords"

In [6]:
exist_Qdrant_Keys = Qdrant.from_existing_collection(
    embedding = embeddings_bkai,
    url = URL_QDRANT_2,
    api_key = API_QDRANT_2,
    prefer_grpc=True,
    collection_name = "semantic_Luat_bkai_More_Keywords",
	metadata_payload_key="metadata"
)

## 2.3. Generate similarity queries

In [7]:
from apikeys_GEMINI import APIKeyManager

from langchain.load import dumps, loads
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

In [8]:
load_dotenv()

MODEL_GEMINI = os.getenv("MODEL_GEMIMI")
# MODEL_GEMINI = "models/gemini-1.5-pro"
if MODEL_GEMINI is None:
    raise ValueError("Environment variable MODEL_GEMINI is not set")
elif not MODEL_GEMINI.startswith("models/"):
    MODEL_GEMINI = f"models/{MODEL_GEMINI}"

APIS_GEMINI_LIST = os.getenv('APIS_GEMINI_LIST').split(',')
key_manager = APIKeyManager(APIS_GEMINI_LIST)

TOP_K = 5
MAX_DOCS_FOR_CONTENT = 5

In [9]:
def query_generator(original_query: str, key_manager) -> list[str]:
    """Generate queries from original query"""
    # Câu truy vấn gốc
    query = original_query
    
    # Cập nhật prompt để yêu cầu rõ ràng chỉ trả về 3 câu truy vấn và câu gốc
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "Bạn là một trợ lý hữu ích, có nhiệm vụ tạo ra nhiều truy vấn tìm kiếm dựa trên một truy vấn gốc."),
            ("human", "Tạo 3 truy vấn tìm kiếm liên quan đến: {original_query}. Mỗi truy vấn trên một dòng mới, và đảm bảo chỉ trả về các truy vấn, không có thêm bất kỳ giải thích hay văn bản nào khác."),
        ]
    )
    
    model = ChatGoogleGenerativeAI(
        # key api google gemini, nếu test mà bị báo lỗi api core thì lấy api khác trong .env để test
        google_api_key=key_manager.get_next_key(),
        model=MODEL_GEMINI,
        temperature=0.1
    )
    
    query_generator_chain = (
        prompt | model | StrOutputParser()
    )
    
    # Kết quả sẽ là một chuỗi các câu truy vấn cách nhau bằng dấu xuống dòng
    result = query_generator_chain.invoke({"original_query": query})
    
    # Tách kết quả thành danh sách các câu truy vấn
    generated_queries = result.strip().split('\n')
    
    # Đảm bảo chỉ lấy 3 câu nếu có nhiều hơn 3 câu truy vấn sinh ra
    if len(generated_queries) > 3:
        generated_queries = generated_queries[:3]
    
    # Kết hợp câu gốc với các câu truy vấn sinh ra
    queries = [query] + generated_queries
    
    return queries

## 2.4. Re-ranker

### 2.4.1. Reciprocal rank fusion (RRF)

In [10]:
def reciprocal_rank_fusion(results: list[list], k=60):
    """Rerank docs (reciprocal rank fusion)"""
    fused_scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    return [x[0] for x in reranked_results[:MAX_DOCS_FOR_CONTENT]]

### 2.4.2. Tính lại score theo trọng số

In [11]:
### continue ...

## 2.5. Tách keywords

### 2.5.1. Tách keywords

In [12]:
# Hàm trích xuất từ khóa từ văn bản, loại bỏ dấu câu ngoại trừ "/" và "-"
def extract_keywords(text):
    text = re.sub(r'[^\w\s/-]', '', text)  # Loại bỏ các dấu câu không mong muốn
    words = text.split()                   # Tách các từ theo khoảng trắng
    unique_words = list(dict.fromkeys(words))  # Loại bỏ từ trùng lặp
    return unique_words

### 2.5.2. Chuyển keywords thành lower case

In [13]:
# Hàm xử lý các từ khóa: chuyển về chữ thường, giữ nguyên số La Mã và từ khóa có "/" hoặc "-"
def process_keywords(keywords):
    result = []
    roman_numeral_pattern = re.compile(r"^(?=[MDCLXVI])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$")
    
    for word in keywords:
        if roman_numeral_pattern.match(word) or any(c in word for c in "/-"):
            result.append(word)
        else:
            result.append(word.lower())  # Chuyển các từ còn lại thành chữ thường
    return result

## 2.6. Truy vấn

In [14]:
TOP_K = 5
MAX_DOCS_FOR_CONTENT = 5

# Câu truy vấn được lấy từ Khoản 2, Điều 8. Người đại diện của người bệnh
# trong tài liệu "Khám bệnh, chữa bệnh", STT: 6

### 2.6.1. Truy vấn cơ bản

#### 2.6.1.1. Với bộ collection semantic chunker

In [15]:
# Query của người dùng
user_Query = "Trường hợp nào thì tôi được chứng minh là thân nhân của người bệnh?"

results = exist_Qdrant_noKeys.similarity_search_with_score(
    query = user_Query,
    filter = None,
    k = TOP_K
)

for doc,score in results:
    print("Page_Content:\n",doc.page_content)
    # print("Context:\n",doc.metadata['context'],"\n")
    print("Stt:",doc.metadata['stt'])
    print("Số hiệu:",doc.metadata['so_hieu'])
    print("Chủ đề:",doc.metadata['chu_de'])
    print("Score:",score,"\n","-----------------------")

Page_Content:
 Chính phủ quy định chi tiết Điều này; quy định điều kiện cấp giấy phép hành nghề quy định tại các khoản 6, 7 và 8 Điều này.
Stt: 6
Số hiệu: ['15-2023-QH15', '15/2023/QH15']
Chủ đề: Khám bệnh, chữa bệnh
Score: 0.30012422800064087 
 -----------------------
Page_Content:
 4. Mức trợ cấp hằng tháng quy định tại khoản 3 Điều này được áp dụng việc điều chỉnh theo quy định tại Điều 67 của Luật này. 5. Trường hợp người đang hưởng trợ cấp hằng tháng chết thì thân nhân được hưởng trợ cấp một lần cho những tháng chưa nhận và được hưởng một lần trợ cấp mai táng nếu đủ điều kiện quy định tại điểm a khoản 1 Điều 85 hoặc điểm a khoản 1 Điều 109 của Luật này.
Stt: 29
Số hiệu: ['41-2024-QH15', '41/2024/QH15']
Chủ đề: Bảo hiểm xã hội
Score: 0.2879490256309509 
 -----------------------
Page_Content:
 Chương III

CHỮ KÝ ĐIỆN TỬ VÀ DỊCH VỤ TIN CẬY

Mục 1.
Stt: 10
Số hiệu: ['20-2023-QH15', '20/2023/QH15']
Chủ đề: Giao dịch điện tử
Score: 0.2847985029220581 
 -----------------------
Page_Conte

#### 2.6.1.2. Với bộ collection keywords

In [16]:
# Query của người dùng
user_Query = "Trường hợp nào thì tôi được chứng minh là thân nhân của người bệnh?"

results = exist_Qdrant_Keys.similarity_search_with_score(
    query = user_Query,
    filter = None,
    k = TOP_K
)

def print_Results_MoreKeywords(results):
    for doc,score in results:
        print("Page_Content:\n",doc.page_content)
        # print("Context:\n",doc.metadata['context'],"\n")
        print("Stt:",doc.metadata['stt'])
        print("Số hiệu:",doc.metadata['so_hieu'])
        print("Chủ đề:",doc.metadata['chu_de'])
        print("Chương:",doc.metadata["Chapter"])
        print("Mục:",doc.metadata["Section"])
        print("Tiểu mục:",doc.metadata["Mini-Section"])
        print("Điều:",doc.metadata["Article"])
        print("Khoản:",doc.metadata["Article-Section"])
        print("Score:",score,"\n","-----------------------")


print_Results_MoreKeywords(results)


Page_Content:
 Điều 110. Giá dịch vụ khám bệnh, chữa bệnh
10. Chính phủ quy định chi tiết Điều này.
Stt: 6
Số hiệu: 15/2023/QH15
Chủ đề: Khám bệnh, chữa bệnh
Chương: Chương X: ĐIỀU KIỆN BẢO ĐẢM HOẠT ĐỘNG KHÁM BỆNH, CHỮA BỆNH
Mục: Mục 2. ĐIỀU KIỆN BẢO ĐẢM VỀ TÀI CHÍNH
Tiểu mục: None
Điều: Điều 110. Giá dịch vụ khám bệnh, chữa bệnh
Khoản: Khoản 10
Score: 0.39879313111305237 
 -----------------------
Page_Content:
 Điều 110. Giá dịch vụ khám bệnh, chữa bệnh
9. Cơ sở khám bệnh, chữa bệnh được đầu tư theo phương thức đối tác công tư quyết định giá dịch vụ khám bệnh, chữa bệnh theo pháp luật về đầu tư theo phương thức đối tác công tư.
Stt: 6
Số hiệu: 15/2023/QH15
Chủ đề: Khám bệnh, chữa bệnh
Chương: Chương X: ĐIỀU KIỆN BẢO ĐẢM HOẠT ĐỘNG KHÁM BỆNH, CHỮA BỆNH
Mục: Mục 2. ĐIỀU KIỆN BẢO ĐẢM VỀ TÀI CHÍNH
Tiểu mục: None
Điều: Điều 110. Giá dịch vụ khám bệnh, chữa bệnh
Khoản: Khoản 9
Score: 0.36634236574172974 
 -----------------------
Page_Content:
 Điều 8. Nghĩa vụ tham gia Công an nhân dân
2. Th

#### **Nhận xét**
- Đối với Collection được xử lý bằng Semantic Chunker (chia theo ngữ nghĩa):
    + Độ chính xác vô cùng thấp.
    + Có lẽ vì lúc thực hiện chunk đã thực hiện chia tách tài liệu không theo cấu trúc mà theo ngữ nghĩa nên độ lớn văn bản quá dài, dẫn đến việc tính toán độ đo Cosin không được tốt.
- Đối với Collection được xử lý bằng Cấu trúc (theo mức Khoản của Điều):
    + Độ chính xác vẫn còn thấp.
    + Tuy đã có sự xuất hiện của file đích nhiều hơn nhưng nội dung vẫn còn sai lệch lớn.

### 2.6.2. Truy vấn có dùng filter (sử dụng keywords)
- Việc sử dụng bộ lọc (**filter**) nhằm gia tăng thêm độ chính xác cho các kết quả tìm kiếm thông qua việc tạo keywords từ các metadata của documents. Vì thế ở phần test này không sử dụng **collection semantic chunker** (tại vì nội dung được phân tách quá lớn nên có lẽ sử dụng phương pháp keywords là không hợp lý).

In [17]:
def create_should_filter(user_keywords, metadata_fields):
    should_conditions = []

    for keyword in user_keywords:
        for field in metadata_fields:
            should_conditions.append(FieldCondition(
                key=field, 
                match=MatchValue(value=keyword)
            ))

    # Trả về bộ lọc với các điều kiện `should`
    return Filter(
        should=should_conditions
    )

In [18]:
def search_documents_with_should_filter(user_query, metadata_fields, top_k=5):
    # Tách keywords từ query của user
    user_keywords = process_keywords(extract_keywords(user_query))
    
    # Tạo bộ lọc `should`
    filter_conditions = create_should_filter(user_keywords, metadata_fields)
    
    # Thực hiện tìm kiếm trên Qdrant với filter `should`
    search_results = exist_Qdrant_Keys.similarity_search_with_score(
        query=user_query,
        filter=filter_conditions,
        k=top_k
    )
    
    return search_results

In [19]:
# Các metadata fields cần lọc
metadata_fields = [
    "metadata.loai_van_ban_Keywords", 
    "metadata.noi_ban_hanh_Keywords", 
    "metadata.so_hieu", 
    "metadata.linhvuc_nganh_Keywords", 
    "metadata.ngay_ban_hanh", 
    "metadata.ngay_hieu_luc", 
    "metadata.chu_de_Keywords", 
    "metadata.key_words", 
    "metadata.Chapter_Keywords", 
    "metadata.Section_Keywords",  
    "metadata.Mini-Section_Keywords", 
    "metadata.Article_Keywords", 
    "metadata.Content_Keywords", 
    "metadata.Article-Section_Keywords"
]

# Query từ người dùng
user_query= "Muốn trở thành người đại diện cho bệnh nhân thì người đó phải có năng lực hành vi dân sự đầy đủ. Vậy các năng lực hành vi dân sự đầy đủ là gì?"

In [20]:
results = search_documents_with_should_filter(user_query, metadata_fields, 5)

print_Results_MoreKeywords(results)

Page_Content:
 Điều 8. Người đại diện của người bệnh
2. Người đại diện của người bệnh phải là người có năng lực hành vi dân sự đầy đủ, bao gồm:
a) Người do người bệnh là người thành niên tự lựa chọn;
b) Người do thành viên gia đình của người bệnh lựa chọn trong trường hợp người bệnh là người thành niên không thể tự lựa chọn và không có ủy quyền trước khi rơi vào tình trạng không thể hoặc có khó khăn trong nhận thức, làm chủ hành vi;
c) Người đại diện theo ủy quyền và người đại diện theo pháp luật của người bệnh theo quy định của Bộ luật Dân sự;
d) Người đại diện theo pháp luật của pháp nhân theo quy định của Bộ luật Dân sự hoặc người được pháp nhân phân công mà pháp nhân đó chịu trách nhiệm quản lý, chăm sóc, nuôi dưỡng người bệnh theo quy định của Bộ luật Dân sự;
đ) Người không thuộc đối tượng quy định tại các điểm a, b, c và d khoản này nhưng tự nguyện thực hiện nghĩa vụ của người bệnh theo quy định của Bộ luật Dân sự.
Stt: 6
Số hiệu: 15/2023/QH15
Chủ đề: Khám bệnh, chữa bệnh
Chương:

#### **Nhận xét**
- Mặc dù kết quả score khá thấp nhưng top1 chính là kết quả chính xác nhất muốn tìm
- Tốc độ phản hồi (response) khá lâu, tại vì phải filter trên 1 số lượng lớn các documents để sắp xếp và lọc ra các lựa chọn phù hợp nhất.

**Như vậy, có thể cho rằng dùng Filter sẽ tốt hơn khi chỉ truy vấn bình thường, bù lại tốc độ phản hồi sẽ hơi lâu một chút**

### 2.6.3. Truy vấn dùng similarity queries (từ Gemini)
- Sinh ra thêm 3 câu hỏi, kết hợp câu hỏi gốc ta được 4 câu
- Dùng nó ta có được 20 kết quả trả về
- Kết quả cuối cùng chỉ lấy ra top5 tài liệu có score cao nhất

In [21]:
def combined_search_and_sort(user_query: str, key_manager, metadata_fields, top_k=5):    
    # Gọi hàm query_generator để sinh ra 3 truy vấn từ query gốc
    queries = query_generator(user_query, key_manager)

    # Lưu trữ tất cả kết quả từ các truy vấn
    all_results = []

    # Thực hiện tìm kiếm cho mỗi query trong danh sách queries
    for query in queries:
        search_results = search_documents_with_should_filter(query, metadata_fields, top_k=top_k)
        all_results.extend(search_results)  # Thêm kết quả vào danh sách

    # Sắp xếp kết quả theo score giảm dần
    sorted_results = sorted(all_results, key=lambda x: x[1], reverse=True)  # x[1] là score

    # Lấy top 5 tài liệu có score cao nhất
    top_results = sorted_results[:top_k]
    
    return top_results


In [22]:
user_query= "Muốn trở thành người đại diện cho bệnh nhân thì người đó phải có năng lực hành vi dân sự đầy đủ. Vậy các năng lực hành vi dân sự đầy đủ là gì?"

results = combined_search_and_sort(user_query, key_manager, metadata_fields, top_k=5)

print_Results_MoreKeywords(results)

Page_Content:
 Điều 8. Người đại diện của người bệnh
2. Người đại diện của người bệnh phải là người có năng lực hành vi dân sự đầy đủ, bao gồm:
a) Người do người bệnh là người thành niên tự lựa chọn;
b) Người do thành viên gia đình của người bệnh lựa chọn trong trường hợp người bệnh là người thành niên không thể tự lựa chọn và không có ủy quyền trước khi rơi vào tình trạng không thể hoặc có khó khăn trong nhận thức, làm chủ hành vi;
c) Người đại diện theo ủy quyền và người đại diện theo pháp luật của người bệnh theo quy định của Bộ luật Dân sự;
d) Người đại diện theo pháp luật của pháp nhân theo quy định của Bộ luật Dân sự hoặc người được pháp nhân phân công mà pháp nhân đó chịu trách nhiệm quản lý, chăm sóc, nuôi dưỡng người bệnh theo quy định của Bộ luật Dân sự;
đ) Người không thuộc đối tượng quy định tại các điểm a, b, c và d khoản này nhưng tự nguyện thực hiện nghĩa vụ của người bệnh theo quy định của Bộ luật Dân sự.
Stt: 6
Số hiệu: 15/2023/QH15
Chủ đề: Khám bệnh, chữa bệnh
Chương:

In [23]:
"""
    Câu truy vấn sau được lấy từ Khoản 3, Điều 2,
    trong tài liệu "Căn cước", stt: 14
"""

user_query= "Giải thích cho tôi hiểu thế nào là \"sinh trắc học\"?"

results = combined_search_and_sort(user_query, key_manager, metadata_fields, top_k=5)

print_Results_MoreKeywords(results)

Page_Content:
 Điều 3. Giải thích từ ngữ
3. Sinh trắc học là những thuộc tính vật lý, đặc điểm sinh học cá biệt và ổn định của một người để nhận diện, phân biệt người này với người khác.
Stt: 14
Số hiệu: 26/2023/QH15
Chủ đề: Căn cước
Chương: Chương I: NHỮNG QUY ĐỊNH CHUNG
Mục: None
Tiểu mục: None
Điều: Điều 3. Giải thích từ ngữ
Khoản: Khoản 3
Score: 0.37720492482185364 
 -----------------------
Page_Content:
 Điều 152. Quy định chuyển tiếp
5. Các Tòa án tiếp tục thực hiện thẩm quyền xét xử sơ thẩm vụ án hành chính; xét xử sơ thẩm, giải quyết vụ việc về sở hữu trí tuệ; giải quyết vụ việc phá sản theo quy định của pháp luật về tố tụng, pháp luật về phá sản cho đến khi các Tòa án nhân dân sơ thẩm chuyên biệt được thành lập và hoạt động theo quy định của luật.
Stt: 21
Số hiệu: 34/2024/QH15
Chủ đề: Tổ chức Tòa án nhân dân
Chương: Chương IX: ĐIỀU KHOẢN THI HÀNH
Mục: None
Tiểu mục: None
Điều: Điều 152. Quy định chuyển tiếp
Khoản: Khoản 5
Score: 0.3297330141067505 
 -----------------------
Pag

#### **Nhận xét**
- Kết quả top đầu chính xác là kết quả muốn có được
- Nhưng thời gian càng tăng khi càng nhiều truy vấn được tạo ra và độ dài, chi tiết ngữ cảnh của truy vấn

**Như vậy, có thể hiểu rằng độ chi tiết của câu hỏi truy vấn càng nhiều, kết quả có thể càng chính xác. Nhưng thay vào đó sẽ mất thời gian hơn trong việc xác định các kết quả trả về tốt nhất**

### 2.6.4. Truy vấn dùng Re-rank ở bước cuối cùng để xếp hạng tài liệu

#### a. Sử dụng hàm Reciprocal rank fusion (RRF)

In [24]:
def combined_search_with_rerank(user_query: str, key_manager, metadata_fields, top_k=5):
    # Gọi hàm query_generator để sinh ra 3 truy vấn từ query gốc
    queries = query_generator(user_query, key_manager)

    # Lưu trữ kết quả cho từng query
    query_results = []

    # Thực hiện tìm kiếm cho mỗi query trong danh sách queries
    for query in queries:
        search_results = search_documents_with_should_filter(query, metadata_fields, top_k=top_k)
        query_results.append(search_results)  # Lưu kết quả riêng cho từng query

    # Sử dụng hàm reciprocal_rank_fusion để re-rank kết quả từ nhiều query
    reranked_results = reciprocal_rank_fusion(query_results)

    # Lấy top 5 kết quả sau khi re-rank
    top_results = reranked_results[:top_k]

    return top_results

In [25]:

# Ví dụ sử dụng
user_query= "Muốn trở thành người đại diện cho bệnh nhân thì người đó phải có năng lực hành vi dân sự đầy đủ. Vậy các năng lực hành vi dân sự đầy đủ là gì?"

results = combined_search_with_rerank(user_query, key_manager, metadata_fields, top_k=5)

# In ra kết quả top 5 tài liệu
print_Results_MoreKeywords(results)


Page_Content:
 Điều 8. Người đại diện của người bệnh
2. Người đại diện của người bệnh phải là người có năng lực hành vi dân sự đầy đủ, bao gồm:
a) Người do người bệnh là người thành niên tự lựa chọn;
b) Người do thành viên gia đình của người bệnh lựa chọn trong trường hợp người bệnh là người thành niên không thể tự lựa chọn và không có ủy quyền trước khi rơi vào tình trạng không thể hoặc có khó khăn trong nhận thức, làm chủ hành vi;
c) Người đại diện theo ủy quyền và người đại diện theo pháp luật của người bệnh theo quy định của Bộ luật Dân sự;
d) Người đại diện theo pháp luật của pháp nhân theo quy định của Bộ luật Dân sự hoặc người được pháp nhân phân công mà pháp nhân đó chịu trách nhiệm quản lý, chăm sóc, nuôi dưỡng người bệnh theo quy định của Bộ luật Dân sự;
đ) Người không thuộc đối tượng quy định tại các điểm a, b, c và d khoản này nhưng tự nguyện thực hiện nghĩa vụ của người bệnh theo quy định của Bộ luật Dân sự.
Stt: 6
Số hiệu: 15/2023/QH15
Chủ đề: Khám bệnh, chữa bệnh
Chương:

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21308\3142861224.py:12: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  (loads(doc), score)


In [26]:

# Ví dụ sử dụng
user_query= "Bảo hiểm xã hội tự nguyện có các chế độ nào?"

results = combined_search_with_rerank(user_query, key_manager, metadata_fields, top_k=5)

# In ra kết quả top 5 tài liệu
print_Results_MoreKeywords(results)


Page_Content:
 Điều 4. Loại hình, các chế độ bảo hiểm xã hội
3. Bảo hiểm xã hội tự nguyện có các chế độ sau đây:
a) Trợ cấp thai sản;
b) Hưu trí;
c) Tử tuất;
d) Bảo hiểm tai nạn lao động theo quy định của Luật An toàn, vệ sinh lao động.
Stt: 29
Số hiệu: 41/2024/QH15
Chủ đề: Bảo hiểm xã hội
Chương: Chương I: NHỮNG QUY ĐỊNH CHUNG
Mục: None
Tiểu mục: None
Điều: Điều 4. Loại hình, các chế độ bảo hiểm xã hội
Khoản: Khoản 3
Score: 0.6678814888000488 
 -----------------------
Page_Content:
 Điều 2. Đối tượng tham gia bảo hiểm xã hội bắt buộc và bảo hiểm xã hội tự nguyện
6. Ủy ban Thường vụ Quốc hội quyết định việc tham gia bảo hiểm xã hội bắt buộc đối với đối tượng khác ngoài đối tượng quy định tại khoản 1 Điều này mà có việc làm, thu nhập ổn định, thường xuyên trên cơ sở đề xuất của Chính phủ phù hợp với điều kiện phát triển kinh tế - xã hội từng thời kỳ.
Stt: 29
Số hiệu: 41/2024/QH15
Chủ đề: Bảo hiểm xã hội
Chương: Chương I: NHỮNG QUY ĐỊNH CHUNG
Mục: None
Tiểu mục: None
Điều: Điều 2. Đối tư

#### **Nhận xét**
- Nhìn chung, việc áp dụng đã thay đổi thứ tự xếp hạng, khiến một số tài liệu có điểm Cosin cao hơn lại ở vị trí thấp hơn.
- Nhưng vẫn có thể áp dụng để đề phòng một số trường hợp kết quả truy vấn bị lệch quá xa, nằm ở top dưới.